# **Sparse Vector**
The sparse vector technique (SVT) takes a stream of queries and a predefined public threshold T. It returns the identities (not values) of the first k queries that are likely larger than the threshold, and nothing else. No matter how many queries are passed to the Sparse Vector Technique, it has a fixed total privacy cost.

**Definitions**

* ε: also known as the privacy budget, and it controls the level of privacy - the smaller the ε, the more privacy is guaranteed. 

* Threshold: the value that is used to determine whether a sensitive data item should be included in an analysis or not.

* Sensitivity: the maximum amount that the output of a computation can change when a single individual's data is added or removed from the input data set. 

* Laplace mechanism: used to add noise to a function's ouput, and the idea is to add the random noise drawn from the Laplace distribution to the function's output with the scale parameter. The amount added is determined by the desired privacy level and the sensitivity of the function.


In [2]:
# import libraries and csv

import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

df = pd.read_csv('adult_with_pii.csv')
df.head()

,Name,DOB,SSN,Zip,Age,Workclass,fnlwgt,Education,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Target
0,Karrie Trusslove,9/7/1967,732-14-6110,64152.0,39.0,State-gov,77516.0,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,Brandise Tripony,6/7/1988,150-19-2766,61523.0,50.0,Self-emp-not-inc,83311.0,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,Brenn McNeely,8/6/1991,725-59-9860,95668.0,38.0,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,Dorry Poter,4/6/2009,659-57-4974,25503.0,53.0,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,Dick Honnan,9/16/1951,220-93-3811,75387.0,28.0,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


# Above Threshold SVT

This can be considered the most basic implementation of the SVT. The implementation is as follows: 
A stream of sensitivity-1 queries, a dataset *D*, a threshold *T*, and the privacy parameter *ε* are inputed to the algorithm.

The goal of this algorithm is to return the index of the first query in queries that exceeds the threshold.

The following is an implementation of the most vasic form of the sparse vector technique. The algorithm takes in sensitivity-1 queries, a dataset, a threshold, and the privacy parameter.



In [3]:
# preserves epsilon-differential privacy
def above_threshold(queries, df, T, epsilon):
    T_hat = T + np.random.laplace(loc=0, scale = 2/epsilon) # T_hat is the noisy threshold 
    for idx, q in enumerate(queries):
        nu_i = np.random.laplace(loc=0, scale = 4/epsilon)
        if q(df) + nu_i >= T_hat: # The noisy query answers are compared to the noisy threshold
            return idx # Algorithm returns the first index that succeeds
    # The algorithm will return a random index if it fails
    return random.randint(0,len(queries)-1)

def above_threshold_fail_signal(queries, df, T, epsilon):
    T_hat = T + np.random.laplace(loc=0, scale = 2/epsilon)   
    for idx, q in enumerate(queries):
        nu_i = np.random.laplace(loc=0, scale = 4/epsilon)
        if q(df) + nu_i >= T_hat:
            return idx
    # as a "fail" signal it returns an invalid index
    return None

The privacy cost of the above threshold algorithm is just ε. 



# Correct Implementation of SVT

In [4]:
# defining the laplace mechanism, which accepts the queries, the sensitivity, and epsilon

def laplace_mechanism(q, s, e):
    return q + np.random.laplace(loc=0, scale=s / e)

The algorithm runs above threshold once and Laplace mechanism twice, and both use 1/3 the privacy budget. It satisifies ε-privacy because of sequential composition. 

In [5]:
def auto_avg(df, epsilon):
    def create_query(b):
        return lambda df: df.clip(lower=0, upper=b).sum() - df.clip(lower=0, upper=b+1).sum()

    # Construct the stream of queries
    bs = range(1,150000,5)
    queries = [create_query(b) for b in bs]
    
    # Run AboveThreshold, using 1/3 of the privacy budget, to find a good clipping parameter
    epsilon_svt = epsilon / 3
    final_b = bs[above_threshold(queries, df, 0, epsilon_svt)]

    # Compute the noisy sum and noisy count, using 1/3 of the privacy budget for each
    epsilon_sum = epsilon / 3
    epsilon_count = epsilon / 3
    
    noisy_sum = laplace_mechanism(df.clip(lower=0, upper=final_b).sum(), final_b, epsilon_sum)
    noisy_count = laplace_mechanism(len(df), 1, epsilon_count)
    
    return noisy_sum/noisy_count

val = auto_avg(df['Age'], 1)
print(val)

38.48660258280501


# Incorrect Implementation of SVT

In [8]:
def auto_avg(df, epsilon):
    def create_query(b):
        return lambda df: df.clip(lower=0, upper=b).sum() - df.clip(lower=0, upper=b+1).sum()

    # Construct the stream of queries
    print("epsilon under stream of queries")
    print(epsilon)
    bs = range(1,150000,5)
    queries = [create_query(b) for b in bs]
    
    # Run AboveThreshold, using 1/3 of the privacy budget, to find a good clipping parameter
    epsilon_svt = epsilon / 3
    print("epsilon svt")
    print(epsilon_svt)
    final_b = bs[above_threshold(queries, df, 0, epsilon_svt)]

    # Compute the noisy sum and noisy count, using 1/3 of the privacy budget for each
    epsilon_sum = epsilon / 2
    print("eps sum")
    print(epsilon_sum)

    epsilon_count = epsilon / 3
    print("eps count")
    print(epsilon_count)
    
    noisy_sum = laplace_mechanism(df.clip(lower=0, upper=final_b).sum(), final_b, epsilon_sum)
    noisy_count = laplace_mechanism(len(df), 1, epsilon_count)
    
    return noisy_sum/noisy_count

val = auto_avg(df['Age'], 1)
print(val)


epsilon under stream of queries
1
epsilon svt
0.3333333333333333
eps sum
0.5
eps count
0.3333333333333333
38.476631975287745


In the incorrect implementation of SVT, the noise added is not scaling the same way and does not have sequential composition. 

# Citations
---

Papers:

*   https://arxiv.org/pdf/1904.12773.pdf
*   https://arxiv.org/pdf/1805.10277.pdf
*   https://link.springer.com/chapter/10.1007/978-3-540-79228-4_1

All the code above was written using the tutorial found here: https://programming-dp.com/ch10.html

